# 📊 Quadratic Fit Analysis with GUI
This notebook allows you to:
- Upload an Excel file
- Select X and Y columns via dropdowns
- Plot the data
- Fit a quadratic curve
- Display error statistics

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from ipywidgets import FileUpload, Dropdown, VBox, Output, Button
from IPython.display import display
import io

In [ ]:
# GUI file uploader
uploader = FileUpload(accept='.xlsx', multiple=False)
x_dropdown = Dropdown(description='X Column:')
y_dropdown = Dropdown(description='Y Column:')
output = Output()
process_button = Button(description='Process Data')

# Global variable to store dataframe
df = None

# Handle file upload
def on_upload_change(change):
    global df
    if uploader.value:
        uploaded_file = list(uploader.value.values())[0]
        content = uploaded_file['content']
        df = pd.read_excel(io.BytesIO(content), engine='openpyxl')
        x_dropdown.options = df.columns.tolist()
        y_dropdown.options = df.columns.tolist()

# Process and plot data
def on_process_clicked(b):
    with output:
        output.clear_output()
        if df is not None and x_dropdown.value and y_dropdown.value:
            x = df[x_dropdown.value].values
            y = df[y_dropdown.value].values

            # Preliminary plot
            plt.figure(figsize=(12, 5))
            plt.subplot(1, 2, 1)
            plt.scatter(x, y, color='blue', label='Data')
            plt.title('Preliminary Plot')
            plt.xlabel(x_dropdown.value)
            plt.ylabel(y_dropdown.value)
            plt.legend()

            # Fit quadratic model
            coeffs = np.polyfit(x, y, 2)
            y_pred = np.polyval(coeffs, x)

            # Fitted plot
            plt.subplot(1, 2, 2)
            plt.scatter(x, y, color='blue', label='Data')
            plt.plot(x, y_pred, color='red', label='Quadratic Fit')
            plt.title('Quadratic Fit')
            plt.xlabel(x_dropdown.value)
            plt.ylabel(y_dropdown.value)
            plt.legend()
            plt.tight_layout()
            plt.show()

            # Error statistics
            mse = mean_squared_error(y, y_pred)
            r2 = r2_score(y, y_pred)
            print(f"Quadratic Coefficients: a={coeffs[0]:.4f}, b={coeffs[1]:.4f}, c={coeffs[2]:.4f}")
            print(f"Mean Squared Error (MSE): {mse:.4f}")
            print(f"R² Score: {r2:.4f}")
        else:
            print("Please upload a file and select X and Y columns.")

# Attach event handlers
uploader.observe(on_upload_change, names='value')
process_button.on_click(on_process_clicked)

# Display GUI
display(VBox([uploader, x_dropdown, y_dropdown, process_button, output]))